In [ ]:
# default_exp problem_types.multi_cls
%load_ext autoreload
%autoreload 2
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
# hide
# test setup
import tensorflow as tf
import numpy as np
from m3tl.test_base import TestBase
from m3tl.input_fn import train_eval_input_fn
from m3tl.test_base import test_top_layer
test_base = TestBase()
params = test_base.params

hidden_dim = params.bert_config.hidden_size

train_dataset = train_eval_input_fn(params=params)
one_batch = next(train_dataset.as_numpy_iterator())


Adding new problem weibo_fake_ner, problem type: seq_tag
Adding new problem weibo_cws, problem type: seq_tag
Adding new problem weibo_fake_multi_cls, problem type: multi_cls
Adding new problem weibo_fake_cls, problem type: cls
Adding new problem weibo_masklm, problem type: masklm
Adding new problem weibo_pretrain, problem type: pretrain
Adding new problem weibo_fake_regression, problem type: regression
Adding new problem weibo_fake_vector_fit, problem type: vector_fit
Adding new problem weibo_premask_mlm, problem type: premask_mlm


/data/m3tl/m3tl/read_write_tfrecord.py:83: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  elif np.issubdtype(type(feature), np.float):


INFO:tensorflow:sampling weights: 
INFO:tensorflow:weibo_fake_cls_weibo_fake_ner: 0.07042253521126761
INFO:tensorflow:weibo_fake_multi_cls: 0.07042253521126761
INFO:tensorflow:weibo_masklm: 0.06690140845070422
INFO:tensorflow:weibo_pretrain: 0.7922535211267606


# Multi-label classification(multi_cls)

Multi-label classification. By default this problem will use `[CLS]` token embedding.

Example: `m3tl.predefined_problems.get_weibo_fake_multi_cls_fn`.

## Imports and utils


In [ ]:
# export
import pickle
from typing import List

import numpy as np
import tensorflow as tf
from m3tl.base_params import BaseParams
from m3tl.problem_types.utils import (empty_tensor_handling_loss,
                                      nan_loss_handling)
from m3tl.special_tokens import PREDICT, TRAIN
from m3tl.utils import (LabelEncoder, get_label_encoder_save_path, get_phase,
                        need_make_label_encoder, variable_summaries)
from sklearn.preprocessing import MultiLabelBinarizer


## Top Layer

In [ ]:
# export

class MultiLabelClassification(tf.keras.Model):
    def __init__(self, params: BaseParams, problem_name: str) -> None:
        super(MultiLabelClassification, self).__init__(name=problem_name)
        self.params = params
        self.problem_name = problem_name
        self.dense = tf.keras.layers.Dense(
            self.params.get_problem_info(problem=problem_name, info_name='num_classes'))
        self.dropout = tf.keras.layers.Dropout(
            1-self.params.dropout_keep_prob
        )
        # self.metric_fn = tfa.metrics.F1Score(
        #     num_classes=self.params.num_classes[problem_name],
        #     threshold=self.params.multi_cls_threshold,
        #     average='macro',
        #     name='{}_f1'.format(problem_name))

    def call(self, inputs):
        mode = get_phase()
        training = (mode == TRAIN)
        feature, hidden_feature = inputs
        hidden_feature = hidden_feature['pooled']
        if mode != PREDICT:
            labels = feature['{}_label_ids'.format(self.problem_name)]
        else:
            labels = None
        hidden_feature = self.dropout(hidden_feature, training)
        logits = self.dense(hidden_feature)

        if self.params.detail_log:
            for weight_variable in self.weights:
                variable_summaries(weight_variable, self.problem_name)

        if mode != PREDICT:
            labels = tf.cast(labels, tf.float32)
            # use weighted loss
            label_weights = self.params.multi_cls_positive_weight

            def _loss_fn_wrapper(x, y, from_logits=True):
                return tf.nn.weighted_cross_entropy_with_logits(x, y, pos_weight=label_weights, name='{}_loss'.format(self.problem_name))
            loss = empty_tensor_handling_loss(
                labels, logits, _loss_fn_wrapper)
            loss = nan_loss_handling(loss)
            self.add_loss(loss)
            # labels = create_dummy_if_empty(labels)
            # logits = create_dummy_if_empty(logits)
            # f1 = self.metric_fn(labels, logits)
            # self.add_metric(f1)

        return tf.nn.sigmoid(
            logits, name='%s_predict' % self.problem_name)

In [ ]:
test_top_layer(MultiLabelClassification, problem='weibo_fake_multi_cls', params=params, sample_features=one_batch, hidden_dim=hidden_dim)

Testing MultiLabelClassification


## Get or make label encoder function


In [ ]:
# export
def multi_cls_get_or_make_label_encoder_fn(params: BaseParams, problem: str, mode: str, label_list: List[str], *args, **kwargs) -> LabelEncoder:

    le_path = get_label_encoder_save_path(params=params, problem=problem)

    if need_make_label_encoder(mode=mode, le_path=le_path, overwrite=kwargs['overwrite']):
        # fit and save label encoder
        label_encoder = MultiLabelBinarizer()
        label_encoder.fit(label_list)
        pickle.dump(label_encoder, open(le_path, 'wb'))
        params.set_problem_info(problem=problem, info_name='num_classes', info=label_encoder.classes_.shape[0])
    else:
        label_encoder = pickle.load(open(le_path, 'rb'))

    return label_encoder

## Label handing function

In [ ]:
# export
def multi_cls_label_handling_fn(target, label_encoder=None, tokenizer=None, decoding_length=None, *args, **kwargs):
    label_id = label_encoder.transform([target])[0]
    label_id = np.int32(label_id)
    return label_id, None

